In [1]:
import flickrapi
import webbrowser

# API keys
API_KEY = "db4c4d646adf494007498f2ffde82ea6"
API_SECRET = "26cad008ca7b167f"

# Code for authenticating the Flickr API
flickr = flickrapi.FlickrAPI(API_KEY, API_SECRET)

if not flickr.token_valid(perms="read"):
    flickr.get_request_token(oauth_callback="oob")
    authorize_url = flickr.auth_url(perms="read")
    webbrowser.open_new_tab(authorize_url)
    verifier = str(input("Verifier code: "))
    flickr.get_access_token(verifier)

In [2]:
import datetime
import pandas as pd
# Get list of photos
curr_time_utc = datetime.datetime.now(datetime.timezone.utc)
five_years_ago_timestamp = (curr_time_utc - datetime.timedelta(days=3*365)).timestamp()
photo_data_filename = "photo_data.csv"
west_coast_bbox = "-124.763068,32.534231,-114.134458,49.002494"
east_coast_bbox = "-87.634938,24.523096,-66.949895,47.459686"
attributes = ["date_upload", "date_taken", "last_update", "owner_name", "geo", "views"]
extracted_attributes = ["dateupload", "datetaken", "lastupdate", "ownername",
                        "latitude", "longitude", "views"]
from os.path import exists
if not exists(photo_data_filename):
    photos_dict = {}
    for attribute in extracted_attributes:
        photos_dict[attribute] = []
    num_pages = 600
    safe_search_level = 1
    has_geo = 1
    for i in range(1, num_pages + 1):
        try:
            photos_metadata = flickr.photos.search(format="parsed-json", min_upload_date=five_years_ago_timestamp,
                                                   min_taken_date=five_years_ago_timestamp, bbox=west_coast_bbox,
                                                   safe_search=safe_search_level, has_geo=has_geo,
                                                   extras=",".join(attributes), page=i)
            photos_metadata2 = flickr.photos.search(format="parsed-json", min_upload_date=five_years_ago_timestamp,
                                                   min_taken_date=five_years_ago_timestamp, bbox=east_coast_bbox,
                                                   safe_search=safe_search_level, has_geo=has_geo,
                                                   extras=",".join(attributes), page=i)
        except flickr.FlickrError:
            continue
        for j in range(len(photos_metadata["photos"]["photo"])):
            for attribute in extracted_attributes:
                photos_dict[attribute].append(photos_metadata["photos"]["photo"][j][attribute])
        for j in range(len(photos_metadata2["photos"]["photo"])):
            for attribute in extracted_attributes:
                photos_dict[attribute].append(photos_metadata2["photos"]["photo"][j][attribute])
    photo_data = pd.DataFrame.from_dict(photos_dict)
    photo_data = photo_data.sample(frac=1).reset_index(drop=True)
    photo_data.to_csv(photo_data_filename, index=False)
else:
    photo_data = pd.read_csv(photo_data_filename)

In [3]:
import reverse_geocoder as rg

loc_data_sample_filename = "location_data.csv"
if not exists(loc_data_sample_filename):
    place_data = photo_data[["latitude", "longitude"]].apply(pd.to_numeric)
    place_data.groupby(["latitude", "longitude"])
    coordinates = [(row["latitude"], row["longitude"]) for _, row in place_data.iterrows()]
    loc_dict = {"latitude": [], "longitude": [], "city": [], "county": [], "state": []}
    locations = rg.search(coordinates)
    for loc in locations:
        loc_dict["latitude"].append(loc["lat"])
        loc_dict["longitude"].append(loc["lon"])
        loc_dict["city"].append(loc["name"])
        loc_dict["county"].append(loc["admin2"])
        loc_dict["state"].append(loc["admin1"])
    loc_data = pd.DataFrame.from_dict(loc_dict)
    loc_data.to_csv(loc_data_sample_filename, index=False)

Loading formatted geocoded file...
